In [1]:
try:
  import torchmetrics
except:
  !pip install -q torchmetrics
  import torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.4 MB/s eta 0:00:00


In [2]:
try:
  import torchinfo
except:
  !pip install -q torchinfo
  import torchinfo

In [3]:
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import one_hot

import os
import random

from torchmetrics import Accuracy
from torchinfo import summary

import PIL
from PIL import Image

import numpy as np

import pathlib as path

import tqdm
from tqdm import tqdm

import torchvision
from torchvision import transforms

import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, imshow

from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [6]:
class Earlystopper:
  def __init__(self, patience = 1, min_delta = 0):

    self.patience = patience
    self.min_delta = min_delta
    self.counter = 0
    self.min_validation_loss = float('inf')

  def earlystop(self, validation_loss):
    if validation_loss < self.min_validation_loss:
      self.min_validation_loss = validation_loss
      self.counter = 0
    elif validation_loss > (self.min_validation_loss + self.min_delta):
      self.counter += 1
      if self.counter >= self.patience:
        return True
    return False

In [7]:
class Conv(nn.Module):
  def __init__(self, in_plane, out_plane, kernel_size = (2,2), stride = 1, padding = 0):
    super(Conv, self).__init__()

    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=in_plane, out_channels=out_plane, kernel_size=kernel_size, stride=stride, padding=padding),
        nn.BatchNorm2d(out_plane, momentum= 0.2),
        nn.Dropout(0.20)

    )

  def forward(self, x: torch.tensor) -> torch.tensor:
    return self.conv(x)

In [8]:
class Dogs_vs_Cats_model(nn.Module):
  def __init__(self):
    super(Dogs_vs_Cats_model, self).__init__()

    self.feature_extractor = nn.Sequential(
      Conv(in_plane=3, out_plane=64, kernel_size=(2,2)),
      Conv(in_plane=64, out_plane=64, kernel_size=(2,2)),
      nn.MaxPool2d((2,2)),
      Conv(in_plane=64, out_plane=128, kernel_size=(2,2)),
      Conv(in_plane=128, out_plane=128, kernel_size=(2,2)),
      nn.MaxPool2d((2,2)),
      Conv(in_plane=128, out_plane=128, kernel_size=(2,2)),

    )


    self.classifier = nn.Sequential(
        nn.Linear(3200, 512),
        nn.Dropout(0.05),
        nn.ReLU(),
        nn.Linear(512, 2)
    )

  def forward(self, x: torch.tensor) -> torch.tensor:

    x = self.feature_extractor(x)
    x = x.view(x.size(0), -1)
    x = self.classifier(x)

    return x

In [9]:
data = torch.rand(1, 3, 32, 32)
model = Dogs_vs_Cats_model()
output = model(data)
output

tensor([[-0.0577,  0.3892]], grad_fn=<AddmmBackward0>)

In [10]:
summary(model, input_size = (1, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
Dogs_vs_Cats_model                       [1, 2]                    --
├─Sequential: 1-1                        [1, 128, 5, 5]            --
│    └─Conv: 2-1                         [1, 64, 31, 31]           --
│    │    └─Sequential: 3-1              [1, 64, 31, 31]           960
│    └─Conv: 2-2                         [1, 64, 30, 30]           --
│    │    └─Sequential: 3-2              [1, 64, 30, 30]           16,576
│    └─MaxPool2d: 2-3                    [1, 64, 15, 15]           --
│    └─Conv: 2-4                         [1, 128, 14, 14]          --
│    │    └─Sequential: 3-3              [1, 128, 14, 14]          33,152
│    └─Conv: 2-5                         [1, 128, 13, 13]          --
│    │    └─Sequential: 3-4              [1, 128, 13, 13]          65,920
│    └─MaxPool2d: 2-6                    [1, 128, 6, 6]            --
│    └─Conv: 2-7                         [1, 128, 5, 5]            --
│ 

In [11]:
class dataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.class_names = ['Cat', 'Dog']
        self.transform = transform

        self.image_paths = []
        self.labels = []

        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(image_dir, class_name)
            image_files = [f for f in os.listdir(class_dir) if f.lower().endswith('.jpg')]
            print(class_dir)
            for image_file in image_files:
                self.image_paths.append(os.path.join(class_dir, image_file))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label_idx = self.labels[idx]
        image = Image.open(image_path).convert('RGB')


        if self.transform:
            image = self.transform(image)
        else:
            transform = transforms.Compose([
                transforms.Resize((32, 32)),
                transforms.ToTensor()
            ])
            image = transform(image)

        return image, label_idx

In [12]:
image_dir = "/content/drive/MyDrive/datasets/PetImages"
loaded_dataset = dataset(image_dir)

/content/drive/MyDrive/datasets/PetImages/Cat
/content/drive/MyDrive/datasets/PetImages/Dog


In [13]:
print(len(loaded_dataset))

24291


In [14]:
train_size = int(0.6 * len(loaded_dataset))
test_size = len(loaded_dataset) - train_size

train_data, test_data = random_split(loaded_dataset, [train_size, test_size])

In [15]:
print(f"train data: {len(train_data)}")
print(f"test data: {len(test_data)}")

train data: 14574
test data: 9717


In [16]:
train_loader = DataLoader(dataset = train_data, batch_size = 32, shuffle= True)
test_loader = DataLoader(dataset = test_data, batch_size = 32, shuffle = True)

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

epochs = 10
model = Dogs_vs_Cats_model().to(device)
criterion = CrossEntropyLoss()
accuracy_fn = Accuracy(task='multiclass', num_classes= 2).to(device)
optimizer = SGD(model.parameters(), lr= 0.002)
scheduler = ReduceLROnPlateau(optimizer= optimizer, mode='min',factor= 0.2, patience= 10, min_lr= 1e-06)
earlystopping = Earlystopper(patience=5, min_delta = 5)

In [ ]:
for epoch in range(epochs):
  model.train()
  running_loss = 0
  running_acc = 0

  train_progress_bar = tqdm(train_loader, total=len(train_loader), desc=f"Epoch: {epoch + 1}/{epochs} - Training",leave= False)

  for x_train, y_train in train_progress_bar:
    x_train = x_train.to(device)
    y_train = y_train.to(device)

    y_logits = model(x_train)
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)

    loss = criterion(y_logits, y_train)
    acc = accuracy_fn(y_pred, y_train)

    running_loss += loss
    running_acc += acc

    Train_loss = running_loss / len(train_loader)
    Train_acc = running_acc / len(train_loader)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  model.eval()
  test_running_loss = 0
  test_running_acc = 0

  test_progress_bar = tqdm(test_loader, total=len(test_loader), desc=f"Epoch: {epoch + 1}/{epochs} - testing",leave= False)

  with torch.inference_mode():
    for x_test, y_test in test_progress_bar:

      x_test = x_test.to(device)
      y_test = y_test.to(device)

      test_y_logits = model(x_test)

      test_y_pred = torch.softmax(test_y_logits, dim=1).argmax(dim=1)

      test_loss= criterion(test_y_logits, y_test)
      test_acc = accuracy_fn(test_y_pred, y_test)

      test_running_loss += test_loss
      test_running_acc += test_acc

      Test_loss = test_running_loss / len(test_loader)
      Test_acc = test_running_acc / len(test_loader)

      if earlystopping.earlystop(test_running_loss):
        break

  print(f"Epoch: {epoch+1}\t|Train Acc: {Train_acc:.02f}%\t|Train Loss: {Train_loss:.02f}%\t|Val Acc: {Test_acc:.02f}%\t|Val Loss: {Test_loss:.02f}%")

Epoch: 1/10 - Training:  83%|████████▎ | 380/456 [35:20<05:38,  4.45s/it]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch: 1	|Train Acc: 0.61%	|Train Loss: 0.66%	|Val Acc: 0.58%	|Val Loss: 0.67%


Epoch: 2	|Train Acc: 0.67%	|Train Loss: 0.60%	|Val Acc: 0.63%	|Val Loss: 0.65%


Epoch: 3	|Train Acc: 0.70%	|Train Loss: 0.58%	|Val Acc: 0.65%	|Val Loss: 0.66%


Epoch: 4	|Train Acc: 0.72%	|Train Loss: 0.55%	|Val Acc: 0.69%	|Val Loss: 0.59%


Epoch: 5	|Train Acc: 0.73%	|Train Loss: 0.54%	|Val Acc: 0.68%	|Val Loss: 0.63%


Epoch: 6	|Train Acc: 0.74%	|Train Loss: 0.53%	|Val Acc: 0.72%	|Val Loss: 0.56%


Epoch: 7/10 - testing:  58%|█████▊    | 176/304 [00:52<00:35,  3.65it/s]

In [ ]:
def visualize_training_result(x1_loss_stat, y1_acc_stat, x2_loss_stat, y2_acc_stat, ):

  plt.figure(figsize= (9,9))

  plt.subplot(2, 2, 1)
  plt.plot(x1_loss_stat)

  plt.subplot(2, 2, 2)
  plt.plot(y1_acc_stat)

  plt.subplot(2, 2, 3)
  plt.plot(x2_loss_stat)

  plt.subplot(2, 2, 4)
  plt.plot(y2_acc_stat)


In [ ]:
model_path = path.Path("model")
model_path.mkdir(parents = True, exist_ok= True)

model_name = "Dog_and_Cat_model"
model_save_path= os.path.join(model_path, model_name)

torch.save(model.state_dict(), model_save_path)

In [ ]:
def save_model():

  model_save_path = "/content/drive/MyDrive/model"
  model_name = "Dog_and_Cat_model_classification"
  model_save_path = os.path.join(model_path, model_name)

  torch.save(model.state_dict(), model_save_path)

In [ ]:
def load_image(path):

  image_path = []
  class_names = ['Cat', 'Dog']
  for class_name in class_names:
    image_dir = os.path.join(path, class_name)
    image_dir_file = [files for files in os.listdir(image_dir) if  files.lower().endswith('jpg')]

    for image in image_dir_file:
      image_full_path = os.path.join(image_dir, image)
      image_path.append(image_full_path)

  return image_path


In [ ]:
def evaluate_model(path, model, num_images=9):
    plt.figure(figsize=(9, 9))

    all_image_path = load_image(path)

    for i in range(num_images):
        idx = random.randint(0, len(all_image_path) - 1)
        image = all_image_path[idx]
        image = Image.open(image).convert('RGB')
        image_data = image
        class_names = ['Cat','Dog']

        transform = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor()
        ])

        image_data = transform(image_data).unsqueeze(dim=0).to(device)

        model.eval()
        with torch.inference_mode():
          prediction = model(image_data)
          predicted_class = prediction.argmax(dim = 1)


        plt.subplot(3, 3, i + 1)
        plt.imshow(np.asarray(image))
        plt.title(class_names[predicted_class])
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
evaluate_model(image_dir, model)